# Start-to-Finish Solution in Keras

Here is my basic method for getting a LB submission churned out. No parameter tuning or data augmentation has been attempted, which should increase the score significantly. 

In [1]:
import os, random
from skimage import io
from skimage.transform import resize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import backend as K

TRAIN_DIR = '../train/'
TEST_DIR = '../test_stg1/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
ROWS = 90  #720
COLS = 160 #1280
CHANNELS = 3

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)
/home/rishikesh/.local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# Loading and Preprocessing Data

Not much processing, other than resizing to 360x640, but you will probably want to run larger images on a GPU for a higher score. I am also keeping track of the labels as I loop through each image folder.  

In [2]:
def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = io.imread(src)
    im = resize(im, (ROWS,COLS), order=3)
    return im


files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

1719 photos of ALB
200 photos of BET
117 photos of DOL
67 photos of LAG
465 photos of NoF
299 photos of OTHER
176 photos of SHARK
734 photos of YFT


In [3]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 3777
Processed 1000 of 3777
Processed 2000 of 3777
Processed 3000 of 3777
(3777, 90, 160, 3)


In [9]:
## Uncomment to check out a fish from each class
#uniq = np.unique(y_all, return_index=True)
# for f, i in zip(uniq[0], uniq[1]):
    #plt.imshow(X_all[i])
    #plt.title(f)
    #plt.show()

# Splitting the Training Data

One-Hot-Encode the labels, then create a stratified train/validation split. 

In [4]:
# One Hot Encoding Labels
y_all = LabelEncoder().fit_transform(y_all)
y_all = np_utils.to_categorical(y_all)

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y_all)

## The Model

Pretty typical CNN in Keras with a plenty of dropout regularization between the fully connected layers. Note: I set the epochs to 1 to avoid timing out - change it to around 20. 

In [5]:
optimizer = RMSprop(lr=1e-4)
objective = 'categorical_crossentropy'

def center_normalize(x):
    return (x - K.mean(x)) / K.std(x)

model = Sequential()

model.add(Activation(activation=center_normalize, input_shape=(ROWS, COLS, CHANNELS)))

model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(FISH_CLASSES)))
model.add(Activation('sigmoid'))

model.compile(loss=objective, optimizer=optimizer)

In [6]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')        
        
model.fit(X_train, y_train, batch_size=64, nb_epoch=20,
              validation_split=0.2, verbose=1, shuffle=True, callbacks=[early_stopping])

Train on 2416 samples, validate on 605 samples
Epoch 1/20
2416/2416 [==============================] - 13s - loss: 1.9161 - val_loss: 1.7788
Epoch 2/20
2416/2416 [==============================] - 13s - loss: 1.8306 - val_loss: 1.8198
Epoch 3/20
2416/2416 [==============================] - 13s - loss: 1.8157 - val_loss: 1.7852
Epoch 4/20
2416/2416 [==============================] - 13s - loss: 1.8056 - val_loss: 1.7898
Epoch 5/20
2416/2416 [==============================] - 13s - loss: 1.8066 - val_loss: 1.7585
Epoch 6/20
2416/2416 [==============================] - 13s - loss: 1.7823 - val_loss: 1.6969
Epoch 7/20
2416/2416 [==============================] - 13s - loss: 1.7863 - val_loss: 1.6607
Epoch 8/20
2416/2416 [==============================] - 13s - loss: 1.7519 - val_loss: 1.6331
Epoch 9/20
2416/2416 [==============================] - 13s - loss: 1.7253 - val_loss: 1.6897
Epoch 10/20
2416/2416 [==============================] - 13s - loss: 1.7064 - val_loss: 1.6151
Epoch 11/20


In [7]:
preds = model.predict(X_valid, verbose=1)
print("Validation Log Loss: {}".format(log_loss(y_valid, preds)))

756/756 [==============================] - 0s     
Validation Log Loss: 1.2596293551029352


# Predicting the Test Set

Finishing off with predictions on the test set.

In [8]:
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)

 992/1000 [============================>.] - ETA: 0s

In [9]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00170.jpg,0.884204,0.203362,0.017741,0.588164,0.238697,0.039009,0.037523,0.528543
1,img_05927.jpg,0.886872,0.312247,0.168433,0.128648,0.669172,0.224466,0.177974,0.361756
2,img_06860.jpg,0.886872,0.312247,0.168433,0.128648,0.669172,0.224466,0.177974,0.361756
3,img_02734.jpg,0.924187,0.172862,0.016748,0.657391,0.405113,0.066585,0.034425,0.194739
4,img_00152.jpg,0.957233,0.222972,0.020450,0.169733,0.025794,0.096475,0.139495,0.921968


In [10]:
sub_file = 'submission_4.csv'
submission.to_csv(sub_file, index=False)